In [89]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [90]:
grain = pd.read_csv('https://byui-cse.github.io/cse450-course/ice/grain/data/grain-training.csv')

In [91]:
grain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3048 entries, 0 to 3047
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3048 non-null   int64  
 1   Perimeter          3048 non-null   float64
 2   Major_Axis_Length  3048 non-null   float64
 3   Minor_Axis_Length  3048 non-null   float64
 4   Eccentricity       3048 non-null   float64
 5   Convex_Area        3048 non-null   int64  
 6   Extent             3048 non-null   float64
 7   Class              3048 non-null   object 
dtypes: float64(5), int64(2), object(1)
memory usage: 190.6+ KB


In [92]:
grain.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,11366,423.114990,171.906647,85.579300,0.867278,11599,0.611404,Osmancik
1,16523,531.892029,224.995422,94.417702,0.907689,16911,0.577041,Cammeo
2,11088,418.208008,172.027420,82.935669,0.876112,11284,0.624993,Osmancik
3,14528,475.447998,192.198563,97.417427,0.862029,14795,0.629490,Cammeo
4,8990,389.377014,157.749603,73.919182,0.883418,9297,0.625261,Osmancik


In [93]:
grain.describe()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent
count,3048.000000,3048.000000,3048.000000,3048.000000,3048.000000,3048.000000,3048.000000
mean,12650.463583,453.910420,188.634550,86.259638,0.886847,12933.628937,0.663072
std,1723.577307,35.418271,17.360515,5.736319,0.020829,1767.781362,0.077284
min,7551.000000,359.100006,145.264465,59.532406,0.777233,7723.000000,0.497413
25%,11357.750000,425.958992,174.235886,82.642729,0.872278,11608.750000,0.599417
50%,12415.000000,448.593506,185.520943,86.279121,0.889010,12681.500000,0.648185
75%,13928.250000,483.222244,203.312111,90.120306,0.902608,14268.500000,0.728501
max,18313.000000,548.445984,238.435089,107.542450,0.948007,18724.000000,0.861050


In [94]:
# b = grain.total.unique()
# print(b)

In [95]:
grain['Class'] = (grain['Class'] == 'Osmancik').astype(int)

In [96]:
grain.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,11366,423.114990,171.906647,85.579300,0.867278,11599,0.611404,1
1,16523,531.892029,224.995422,94.417702,0.907689,16911,0.577041,0
2,11088,418.208008,172.027420,82.935669,0.876112,11284,0.624993,1
3,14528,475.447998,192.198563,97.417427,0.862029,14795,0.629490,0
4,8990,389.377014,157.749603,73.919182,0.883418,9297,0.625261,1


In [97]:
grain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3048 entries, 0 to 3047
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3048 non-null   int64  
 1   Perimeter          3048 non-null   float64
 2   Major_Axis_Length  3048 non-null   float64
 3   Minor_Axis_Length  3048 non-null   float64
 4   Eccentricity       3048 non-null   float64
 5   Convex_Area        3048 non-null   int64  
 6   Extent             3048 non-null   float64
 7   Class              3048 non-null   int32  
dtypes: float64(5), int32(1), int64(2)
memory usage: 178.7 KB


In [98]:
X = grain.drop("Class", axis=1)
y = grain.Class

In [99]:
X

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent
0,11366,423.114990,171.906647,85.579300,0.867278,11599,0.611404
1,16523,531.892029,224.995422,94.417702,0.907689,16911,0.577041
2,11088,418.208008,172.027420,82.935669,0.876112,11284,0.624993
3,14528,475.447998,192.198563,97.417427,0.862029,14795,0.629490
4,8990,389.377014,157.749603,73.919182,0.883418,9297,0.625261
...,...,...,...,...,...,...,...
3043,14078,478.470001,203.645462,88.560310,0.900491,14280,0.744395
3044,17246,540.541992,225.988861,98.573151,0.899857,17704,0.573929
3045,11070,419.403015,173.575043,82.154213,0.880898,11266,0.600586
3046,11747,452.127014,194.494858,78.744461,0.914376,11935,0.542637


In [100]:
y

0       1
1       0
2       1
3       0
4       1
       ..
3043    0
3044    0
3045    1
3046    0
3047    1
Name: Class, Length: 3048, dtype: int32

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [102]:
# Initialize and train the model
model = XGBRegressor()
model.fit(X_train, y_train)

# Make predictions and evaluate the model
predictions = model.predict(X_test)
result = mean_squared_error(y_test, predictions, squared=False)

C:\Users\Surplus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [114]:
# Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV

xgb_params = {
    'objective': ['binary:logistic'],
    'learning_rate': [0.01, 0.03, 0.001],
    'n_estimators': [25, 100, 1000],
    'max_depth': [6, 10, 15],
    'gamma': [0, 0.6],
    'subsample': [0.52, 1],
    'colsample_bytree': [0.6, 1],
    'reg_lambda': [1, 2],
    'booster': ['gbtree'],
}

xgb_base = XGBRegressor(**xgb_params)
xgb_cv = RandomizedSearchCV(xgb_base, xgb_params, cv= 10)
xgb_cv.fit(X_train, y_train)

print("Tuned Desicion XGB Parameters: {}".format(xgb_cv.best_params_))
print("Best score is {}: ".format(xgb_cv.best_score_))

Tuned Desicion XGB Parameters: {'subsample': 0.52, 'reg_lambda': 1, 'objective': 'binary:logistic', 'n_estimators': 100, 'max_depth': 15, 'learning_rate': 0.03, 'gamma': 0, 'colsample_bytree': 1, 'booster': 'gbtree'}
Best score is 0.7678702631179336: 


In [ ]:
# Fix this!

xgb_stack = XGBRegressor(subsample = 0.52, 
                         reg_lambda = 1, 
                         objective = 'binary:logistic', 
                         n_estimators = 100, 
                         max_depth = 15, 
                         learning_rate = 0.03, 
                         gamma = 0, 
                         colsample_bytree = 1, 
                         booster = 'gbtree')
xgb_stack.fit(X_train, y_train)
xgb_stack_train_prediction = xgb_stack.predict(X_train)

In [103]:
predictions = np.where(model.predict(X_test)< 0, 0, 1)

In [104]:
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,

In [105]:
result = mean_squared_error(y_test, predictions, squared=False)
result

C:\Users\Surplus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


0.5340841709312721

In [106]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test,predictions)
r2

-0.16754116754116732

In [107]:
#Accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.7147540983606557

In [108]:
holdout = pd.read_csv('https://byui-cse.github.io/cse450-course/ice/grain/data/grain-holdout.csv')

In [109]:
holdout.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent
0,12899,472.263000,205.520554,80.433365,0.920236,13107,0.585919
1,11693,424.503998,176.024063,84.991318,0.875709,11842,0.606735
2,11329,419.404999,173.390457,84.345825,0.873708,11560,0.803247
3,12278,453.692993,184.050156,86.660088,0.882213,12601,0.629835
4,11231,427.666992,183.587097,78.861732,0.903038,11427,0.721184


In [110]:
predict_holdout = model.predict(holdout)
predict_holdout

array([ 2.01041237e-01,  1.00212848e+00,  1.00400877e+00,  1.02952814e+00,
        7.65067101e-01,  4.01011296e-03,  4.08073872e-01,  1.99716375e-03,
        7.35392570e-01,  1.00599968e+00,  9.68781173e-01,  9.90019858e-01,
       -4.49523842e-03,  4.65222090e-01, -7.51163904e-03,  1.00210130e+00,
        9.81830478e-01,  1.03226328e+00,  1.10148466e+00,  2.13555880e-02,
        7.32366145e-01,  1.98906854e-01,  9.96196628e-01,  1.00677145e+00,
        4.81575578e-01,  2.45080283e-03, -3.67549844e-02,  7.42878079e-01,
        2.06721146e-02,  9.81221907e-03,  2.82436656e-03, -9.14332550e-03,
       -4.79948008e-03,  6.62744999e-01, -2.60381075e-03,  1.91958249e-03,
       -5.75300399e-03,  2.94840639e-03,  9.91190135e-01,  1.80058517e-02,
        1.01034117e+00,  9.85652506e-01,  2.94981133e-02,  1.03939235e+00,
        9.95914936e-01,  4.42724815e-03,  1.02587259e+00,  9.97819901e-01,
        9.91940856e-01, -8.42012838e-03,  8.10002983e-01,  9.88022804e-01,
        4.80036974e-01,  

In [111]:
converted_predictions = np.where(predict_holdout < 0, "Cammeo", "Osmancik")

In [112]:
converted_predictions

array(['Osmancik', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik',
       'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik',
       'Osmancik', 'Osmancik', 'Cammeo', 'Osmancik', 'Cammeo', 'Osmancik',
       'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik',
       'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik',
       'Cammeo', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik', 'Cammeo',
       'Cammeo', 'Osmancik', 'Cammeo', 'Osmancik', 'Cammeo', 'Osmancik',
       'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik',
       'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik', 'Osmancik',
       'Osmancik', 'Cammeo', 'Osmancik', 'Osmancik', 'Osmancik',
       'Osmancik', 'Cammeo', 'Osmancik', 'Osmancik', 'Osmancik',
       'Osmancik', 'Osmancik', 'Osmancik', 'Cammeo', 'Osmancik', 'Cammeo',
       'Osmancik', 'Cammeo', 'Cammeo', 'Osmancik', 'Osmancik', 'Osmancik',
       'Osmancik', 'Cammeo', 'Cammeo', 'Cammeo', 'Osmancik', 'Osmancik',
       'Cammeo', 'Osma

In [113]:
output_file = "duane_siu-ice-grain-predictions.csv"
predict_holdout = pd.DataFrame(converted_predictions, columns=['predictions'])
predict_holdout.to_csv(output_file, index = False)
print(f"Dataset saved to {output_file}. You can download it from here.")

Dataset saved to duane_siu-ice-grain-predictions.csv. You can download it from here.
